In [14]:
import xml.etree.ElementTree as ET, sys, urllib.request
if 'functions' in sys.modules:
    del sys.modules['functions'] # deletes previously imported module so that potential changes can be loaded
from functions import *

In [52]:
# parse XML
def parse_xml(url):
    tree = ET.parse(urllib.request.urlopen(url))
    root = tree.getroot() # root consists of 4 elements: vorspann, sitzungsverlauf, anlagen, rednerliste
    return root

In [53]:
# save redner and their ids
def get_redner_dict(root, redner):
    for r in root.find('rednerliste'):
        try:
            titel = r.find('name').find('titel').text
        except AttributeError:
            titel = ''
        try:
            vorname = r.find('name').find('vorname').text
        except AttributeError:
            vorname = ''
        try:
            nachname = r.find('name').find('nachname').text
        except AttributeError:
            nachname = ''
        try:
            fraktion = ', ' + r.find('name').find('fraktion').text
        except AttributeError:
            fraktion = ''
        redner[r.attrib['id']] = cleanup(' '.join([titel, vorname, nachname+fraktion]).strip())
    return redner

In [86]:
# extract text from 'reden' and save them with their redner_id in a dict
def get_reden(root, d):
#     for i in range(len(root.find('sitzungsverlauf'))):
    for r in root.find('sitzungsverlauf'):
        reden = r.findall('rede')
        if len(reden) == 0:
            continue
#         reden = root[1][i].findall('rede')
        try:
            redner_ids = [[p.find('redner') for p in rede.findall('p') if p.attrib['klasse'] == 'redner'][0].attrib['id'] for rede in reden]
            texte = [' '.join([p.text for p in rede.findall('p') if not p.attrib['klasse'] == 'redner']) for rede in reden]
        except KeyError: # some xml files contain empty elements with no 'klasse'. skip them.
            continue
        for redner_id, text in zip(redner_ids, texte):
            text = cleanup(text)
            if not redner_id in d.keys():
                d[redner_id] = [text]
            else:
                d[redner_id].append(text)
    return d

In [88]:
# parse XML files
urls = ['https://www.bundestag.de/resource/blob/664948/fd846b015600a686747b1a23ba902d0f/19121-data.xml',
        'https://www.bundestag.de/resource/blob/664710/4a1a6d42e4ce3aadd9f9d1d170577512/19120-data.xml',
        'https://www.bundestag.de/resource/blob/663968/2f0583192968310c0b6b44ecf4009a7d/19119-data.xml',
        'https://www.bundestag.de/resource/blob/663594/6d6c892cb2f2bb8a96b21223c64e04e3/19118-data.xml',
        'https://www.bundestag.de/resource/blob/663116/9d19894a5406d322d2edd9e8875ba017/19117-data.xml']
redner, reden = {}, {}
for url in urls:
    root = parse_xml(url)
    redner = get_redner_dict(root, redner)
    reden = get_reden(root, reden)